<a id='section0'></a><br>
# Segmenting and Clustering Neighborhoods in the city of Toronto, Canada

---

<b>[1. Scrape Data](#section1)</b><br>
[1a. The End Result](#section1a)<br>
<br>
<b>[2. Add Geo-spacial Data](#section2)</b><br>
[2a. Get Coordinates in the table with csv](#section2a)<br>
[2b. The End Result](#section2b)<br>
<br>
<b>[3. Explore and cluster the neighborhoods in Toronto](#section3)</b><br>
[3a. Preparation work](#section3a)<br>
[3b. Explore Neighborhoods in Toronto](#section3b)<br>
[3c. Analyze Neihgborhoods in Toronto](#section3c)<br>
[3d. The End Result](#section3d)<br>


---

<a id='section1'></a>
## 1. Scrape Data

The neighborhood data comes from https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M 

Useful links for the scraping:<br>
http://beautiful-soup-4.readthedocs.io/en/latest/<br>
https://www.youtube.com/watch?v=ng2o98k983k

In [1]:
# All imports we might need

import requests
import pandas as pd # for data analysis
#     pd.set_option('display.max_columns', None)
#     pd.set_option('display.max_rows', None)
import numpy as np # handle data in a vectorized manner
import json # to handle json files

#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans
from sklearn.datasets.samples_generator import make_blobs
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
#!conda install -c conda-forge bs4 --yes
from bs4 import BeautifulSoup 
#!conda install -c conda-forge lxml --yes
import lxml

print('All the libraries are imported.')

All the libraries are imported.


In [2]:
# get the table from the webpage
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(url).text
soup = BeautifulSoup(source, 'xml')
table=soup.find('table')

In [3]:
# make our dataframe
cols = ['Postalcode','Borough','Neighborhood']
df = pd.DataFrame(columns = cols)

In [4]:
# fill our dataframe
for tr in table.find_all('tr'):
    row_data=[]
    for td in tr.find_all('td'):
        row_data.append(td.text.strip())
    if len(row_data)==3:
        df.loc[len(df)] = row_data
        #print(row_data)

In [5]:
df.head(5)

,Postalcode,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [6]:
# remove rows where Borough is Not Assigned
df=df[df['Borough']!='Not assigned']

In [7]:
# drop duplicates
df=df.drop_duplicates()

In [8]:
# replace / with ,
df['Neighborhood'] = df['Neighborhood'].str.replace(' /',',')

<a id='section1a'></a>
### 1a. The end-result

In [9]:
df.head(5)

,Postalcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [10]:
df.shape

(103, 3)

<i>[Go back up](#section0)<br></i>

---

<a id='section2'></a>
## 2. Add Geo-spacial Data

<a id='section2a'></a>
### 2a. Get Coordinates in the table with csv

In [11]:
# read the geo data file
df_geo = pd.read_csv('http://cocl.us/Geospatial_data')
df_geo.head(5)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [12]:
# if we rename the postal code column we can merge easy.
df_geo.rename(columns={'Postal Code':'Postalcode'}, inplace = True)
df_geo.set_index = ("PostalCode")
df.set_index("Postalcode")
df_tor = pd.merge(df,df_geo)
df_tor.head(5)

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


<a id='section2b'></a>
### 2b. The end-result

In [13]:
df_tor.head(10)

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


<i>[Go back up](#section0)<br></i>

---

<a id='section3'></a>
## 3. Explore and cluster the neighborhoods in Toronto

<a id='section3a'></a>
### 3a. Preparation work
Get the coordinates of the city of Toronto

In [14]:
from geopy.geocoders import Nominatim # convert addresses into latitude and longitude values

address = 'Toronto ON, Canada'
geolocator = Nominatim(user_agent = 'ua')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The coordinates of Toronto are: {}, {}.'.format(latitude,longitude))

The coordinates of Toronto are: 43.6534817, -79.3839347.


#### Visualize Toronto's neighborhoods on a Map.

In [15]:
import folium # map rendering

# create map of toronto
map_tor = folium.Map(location=[latitude,longitude],zoom_start = 10)

for lat, lng, borough, neighborhood in zip(df_tor['Latitude'],df_tor['Longitude'], df_tor['Borough'], df_tor['Neighborhood']):
    label = '{},{}'.format(neighborhood,borough)
    label = folium.Popup(label,parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_tor)  
    
map_tor


#### Prepare the Foursquare API URL.

Reading the api credentials from a json file.<br>
_If you wish to use your own credentials, to try this out you will have to replace this section with your own API credentials._

In [24]:
#import json

# my foursquare API credentials
with open('credentials.json') as f:
    data = json.load(f)
    CLIENT_ID = data['foursquare_client_id']
    CLIENT_SECRET  = data['foursquare_client_code']
    VERSION = data['foursquare_version']

# use the below code to use your api client detail without the credentials file
#CLIENT_ID = ''
#CLIENT_SECRET = ''
#VERSION = '20180605'

#print(CLIENT_ID)
#print(CLIENT_SECRET)
#print(VERSION)

Verifying if the URL construction before using it.

In [25]:
limit = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    limit)

#url

#### Get the venues from Foursquare

<a id='section3b'></a>
### 3b.  Explore Neighborhoods in Toronto

In [26]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
# https://stackoverflow.com/questions/54244782/how-to-fix-key-error-groups-in-using-fousquare-api-python
toronto_venues = getNearbyVenues(names=df_tor['Neighborhood'],
                                   latitudes=df_tor['Latitude'],
                                   longitudes=df_tor['Longitude']
                                  )

#### Let's have a look at our results.
The size of the dataframe.

In [ ]:
print(toronto_venues.shape)

The result list.

In [ ]:
toronto_venues.head(10)

The amount of different types of venues.

In [ ]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

The amount of venues per neighborhood.

In [ ]:
toronto_venues.groupby('Neighborhood').count()

<a id='section3c'></a>
### 3c. Analyze Each Neighborhood

In [ ]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]



Let's have a look at our result and the dataframe size.<br>
_I removed the restriction for the amount of columns in order to see them all._

In [ ]:
toronto_onehot.shape

In [ ]:
pd.set_option('display.max_columns', None)
toronto_onehot.head(10)

We group the neighborhoods and take them mean frequency of each category of venue.

In [ ]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.shape

In [ ]:
toronto_grouped.head(5)

### Top 3 venues per neighborhood

In [ ]:
num_top_venues = 3

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

Now we need to do the same, but in a dataframe, for the top 5 venues.

For this we need a sorting function:

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)



In [ ]:
toronto_grouped.shape

In [ ]:
neighbourhoods_venues_sorted.head(5)

### Clustering the Neighborhoods with K-means

Creating the clusters

In [ ]:
k = 5
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=k, random_state=0).fit(toronto_grouped_clustering)

Adding the clusters a dataframe.

In [ ]:
neighbourhoods_venues_sorted.insert(0, 'Clusters', kmeans.labels_)

toronto_merged = df_tor

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

In [ ]:
toronto_merged.head(10)

Some neighborhoods, like Islington Avenue, have no information (NaN).<br>
Lets remove these from our dataframe.

In [ ]:
toronto_merged = toronto_merged.dropna()

In [ ]:
toronto_merged.head(10)

Let put all of this in a map now.

In [ ]:
toronto_merged['Clusters'] = toronto_merged.Clusters.astype(int)

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Clusters']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       


<a id='section3d'></a>
### 3d. The End-Result.

In [ ]:
map_clusters

In [ ]:
Analyze Each Neighborhood

<i>[Go back up](#section0)<br></i>

---